In [1]:
# uncomment and runif you aren't already setup.
# %pip install -Uq langchain_community langchain toml pandas datasets chromadb

In [2]:
%pip install -Uq trulens_eval

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

import pandas as pd
import sys



In [4]:
# TODO: cleanup
#import toml
#from langchain_community.chat_models import ChatOpenAI
#from langchain_community.llms import HuggingFaceEndpoint
#from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceInferenceAPIEmbeddings
#from langchain_community.llms.ollama import Ollama

In [5]:
from rag import get_model, setup_rag, setup_retriever

In [7]:
import os
import toml

openai_api_key = None
huggingfacehub_api_token = None

try:
    with open('secrets.toml', 'r') as f:
        config = toml.load(f)
        huggingfacehub_api_token = config['HUGGINGFACEHUB_API_TOKEN']
        openai_api_key = config['OPENAI_API_KEY']
except:
    print("Could not load secrets")

os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingfacehub_api_token
os.environ['OPENAI_API_KEY'] = openai_api_key

In [8]:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [9]:
from rag import get_model, setup_retriever, setup_rag
context_docs = []

def save_context(docs):
    context_docs.append([doc.page_content for doc in docs])
    return docs

model = get_model()
retriever, vs = setup_retriever()
rag_chain = setup_rag(model, retriever, process_docs=save_context)


Opened vector database


In [10]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.middle[0].get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.middle[0].get_relevant_documents.rets .


[nltk_data] Downloading package punkt to /Users/ranton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
def do_trulens_eval():
    testset_path = "testset.csv"
    results_path = "rag_results.json"
    scored_path = "rag_scores.csv"

    testset_df = pd.read_csv(testset_path)

    # TODO: Limit testset size to test more quickly.
    testset_df = testset_df.head(3) 
    
    queries = testset_df['Question']

    model = get_model()
    retriever, vs = setup_retriever()
    rag_chain = setup_rag(model, retriever)

    tru_recorder = TruChain(rag_chain,
    app_id='Chain1_Rag',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

    responses = []
    records = []
    # This could use batch inference.
    n_total = len(queries)
    for i, query in enumerate(queries):
        print(f"Processing query #{i+1} of {n_total}")
        response, tru_record = tru_recorder.with_record(rag_chain.invoke, query)
        responses.append(response)
        records.append(tru_record)

    print("Batch done.")


    records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_Rag"])
    return records, feedback

In [14]:
records, feedback = do_trulens_eval()

Opened vector database
Processing query #1 of 3
Processing query #2 of 3


Groundedness per statement in source:   0%|          | 0/14 [00:00<?, ?it/s]

Processing query #3 of 3


Groundedness per statement in source:   0%|          | 0/13 [00:00<?, ?it/s]

Batch done.


Groundedness per statement in source:   0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,relevance_calls,context_relevance_with_cot_reasons_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_Rag,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_77626154fad9ff17a6987fefa4993518,"""What percentage of patients with chronic GERD...","""## Researching the Prevalence of Barrett's Es...",-,"{""record_id"": ""record_hash_77626154fad9ff17a69...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-05-14T07:01:28.110499"", ""...",2024-05-14T07:01:40.782951,0.2,0.0,0.0,[{'args': {'prompt': 'What percentage of patie...,[{'args': {'question': 'What percentage of pat...,"[{'args': {'source': [[]], 'statement': '## Re...",12,0,0.0
1,Chain1_Rag,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_e9e2d5270e3601df2936c8b43d205a16,"""What is the recommended surveillance interval...","""## Researching the recommended surveillance i...",-,"{""record_id"": ""record_hash_e9e2d5270e3601df293...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-05-14T07:01:41.018201"", ""...",2024-05-14T07:01:53.493323,0.8,0.0,NaN,[{'args': {'prompt': 'What is the recommended ...,[{'args': {'question': 'What is the recommende...,NaN,12,0,0.0
2,Chain1_Rag,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_706b6dd65ced2258227d2fbfdbb88350,"""Which imaging technology is recommended for t...","""## Researching Imaging Technology for Detecti...",-,"{""record_id"": ""record_hash_706b6dd65ced2258227...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-05-14T07:01:53.652732"", ""...",2024-05-14T07:02:04.996227,NaN,NaN,NaN,NaN,NaN,NaN,11,0,0.0


In [16]:
tru.get_leaderboard(app_ids=["Chain1_Rag"])

,context_relevance_with_cot_reasons,relevance,groundedness_measure_with_cot_reasons,latency,total_cost
app_id,,,,,
Chain1_Rag,0.0,0.6,0.0,11.666667,0.0
